# NNTI Assignment 6

Name 1: <br>
Student id 1: <br>
Email 1: <br>

Name 2: <br>
Student id 2:  <br>
Email 2:  <br>

Name 3: <br>
Student id 3:  <br>
Email 3: <br>

**Instructions:**
- Read each question carefully. <br/>
- Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook. <br>
- Upload the zipped folder on CMS. Please follow the naming convention of **Name1_id1_Name2_id2_Name3_id3.zip**. Only one member of the group should make the submisssion.

## Batch Normalization
In this exercise you will again construct a feed forward neural network, however you will use batch normalization during training. Training deep neural networks can be challenging due to the change in the distribution of inputs to layers deep in the network as a result of the updates of the weights in the previous layers. This causes the learning to chase a "moving target", which slows down the learning process. Batch normalization is a technique that aims to address this problem by normalizing layer inputs. This stabilizes the learning process and can greatly decrease training time. If you are interested you can read the paper introducing batch normalization [here](https://arxiv.org/abs/1502.03167).

You will be working with the [FashionMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.FashionMNIST.html) dataset by Zalando, which consists of $28\times 28$ black and white images and has 10 classes just like the [MNIST](https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html) dataset. But instead of numbers the classes are various items of clothing such as shoes, t-shirts, dresses, etc.

### 1. Baseline Network (1 point)
You are provided with a dataloader for the train and test sets, each with a batch size of 64. Your task is to construct a feed forward neural network with 3 hidden linear layers with a ReLU after each of the first 2 layers. The first layer should have a hidden size of 64 and the second a hidden size of 32. This is a multi-class classification problem so you will need to use cross entropy loss (provided by [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)). Use a stochastic gradient descent (provided by [torch.optim.SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)) with a learning rate of 0.001 as the optimizer. Train your network on the training data for 5 epochs and report accuracy on the **test** set after each epoch, for this refer to [PyTorch Training Loop](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html#the-training-loop) and [Per-Epoch Activity](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html#per-epoch-activity). Make sure to save the accuracies.
(hint: the data comes in the format of $28 \times 28$ tensors, so you will need flatten it to train your network)

In [2]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
import matplotlib.pyplot as plt

In [ ]:
# load train and test set
fashion_trainset = torchvision.datasets.FashionMNIST('data/', train=True, download=True, transform=transforms.ToTensor())
fashion_testset = torchvision.datasets.FashionMNIST('data/', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
# get train and test loader
fashion_train_loader = torch.utils.data.DataLoader(dataset=fashion_trainset, batch_size=64, shuffle=True)
fashion_test_loader = torch.utils.data.DataLoader(dataset=fashion_testset, batch_size=64, shuffle=False)

In [ ]:
class FashionNetwork(nn.Module):
    def __init__(self):
        # TODO
        # Fill in the layers provided by PyTorch that correspond to the description.
        # Define the loss function
        # Define the optimizer
        NotImplemented

    def forward(self, x):
        # TODO
        NotImplemented

    def optimize(self):
        # TODO
        # Don't forget to test after each epoch and make sure to save the accuracies
        NotImplemented

    def test(self):
        # TODO
        NotImplemented

In [ ]:
fashion_network = FashionNetwork()
fashion_network.optimize(fashion_train_loader, fashion_test_loader)

### 2. Network with Batch Normalization (0.5 points)
Construct another network with the same parameters as 2.1 but this time include a batch normalization layer (use [nn.BatchNorm1d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)). Where to place this layer is up to you, but you can reference the lecture slides for inspiration. Again train your network for 5 epochs and report **test** accuracy after each epoch.

In [ ]:
class FashionNetworkBatchNorm(nn.Module):
    # TODO
    NotImplemented

In [ ]:
fashion_network_batch_norm = FashionNetworkBatchNorm()
fashion_network_batch_norm.optimize(fashion_train_loader, fashion_test_loader)

### 3. Plotting the Performances (0.25 points)
Plot the accuracies of the 2 networks and **discuss what you observe**.

In [ ]:
# TODO

**Your Answer:**